In [78]:
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.postprocessor.cohere_rerank import CohereRerank
import nest_asyncio
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
import os

In [100]:
# allows nested access to event loop
load_dotenv()
nest_asyncio.apply()


In [101]:
# cohere API key
API_KEY = os.environ.get("API_KEY")
# input doc directory
input_dir_name = os.environ.get("DIR")
print(input_dir_name)

/Users/msrivas/hack/ai/talk2doc/docs


In [102]:
loader = SimpleDirectoryReader(
  input_dir=input_dir_name,
  required_exts=['.pdf'],
  recursive=True
)

docs = loader.load_data()

In [103]:
embed_model = CohereEmbedding(
  cohere_api_key=API_KEY,
  model_name='embed-english-v3.0',
  input_type='search_query'
)

In [104]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

# Create vecotr store and upload indexed data
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs)

In [105]:
from llama_index.llms.cohere import Cohere
from llama_index.postprocessor.cohere_rerank import CohereRerank

# create query engine
cohere_rerank = CohereRerank(api_key=API_KEY, top_n=2)
Settings.llm = Cohere(api_key=API_KEY, model='command-r-plus')
query_engine = index.as_query_engine(node_postprocessors=[cohere_rerank])

In [106]:
from llama_index.core import PromptTemplate
qa_prompt_tmpl_str = (
  "Context information is below.\n"
  "-------------------------\n"
  "{context_str}\n"
  "Given the context information above I want you\n"
  "to think step by step to answer the query in a crisp \n"
  "manner, incase you don't knkow the answer say \n"
  "'I don't know!'. \n"
  "Query: {query_str}\n"
  "Answer: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})
question = 'explain do VAE\'s work in detail. Explain in steps how one would go about building a VAE'
response = query_engine.query(question)

In [ ]:
import textwrap

wrapped_text = textwrap.fill(str(response), width=80)
print(wrapped_text)